In [1]:
import collections
import warnings
from pandas.core.common import SettingWithCopyWarning
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.font_manager
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import forestci as fci
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

Failed to import duecredit due to No module named 'duecredit'


In [2]:
signal = pd.read_csv("resonances0.txt", delim_whitespace=True)
val_data = pd.read_csv("comp.txt", delim_whitespace=True)
signal = signal.replace('-',np.nan)
gamma_test = signal[signal['Gg'].isnull()]
gamma_x_test = gamma_test[['Z','A']]
signal.dropna(inplace=True)

gamma_x_train = signal[['Z','A']]
gamma_y_train = signal[['Gg']]
gamma_x_train = gamma_x_train.values.reshape(-1,2)
gamma_y_train = gamma_y_train.values.ravel()

rfmissgg = RandomForestRegressor(n_estimators=500,max_depth=250,random_state=10,max_features=2)
rfmissgg.fit(gamma_x_train,gamma_y_train)

y_pred = rfmissgg.predict(gamma_x_test)
gamma_x_test['Gg'] = y_pred


In [3]:
gamma_train = signal[['Z','A','Gg']]
gamma_full = pd.concat([gamma_train, gamma_x_test])
gamma_full = gamma_full.sort_values(by=['Z'])


In [4]:
x = gamma_full[['Z','A']]
y = gamma_full[['Gg']]
arr_x = x.values.reshape(-1,2)
arr_y = y.values.ravel()
X_train, X_test, y_train, y_test = train_test_split(arr_x, arr_y, test_size=0.2, random_state=11)

In [5]:
rf = RandomForestRegressor(n_estimators=500,max_depth=500,random_state=10,max_features=2)
rf.fit(X_train,y_train)
rf.score(X_train,y_train)

y_pred = rf.predict(X_test)
rf_err = abs(fci.random_forest_error(rf,X_train,X_test,calibrate=False))

In [6]:
with open("data_gamma.txt","w") as f:
    # f.write(' Z=  22 A=  46\n')
    # f.write('  U[MeV]  fE1[mb/MeV]\n')
    for i in range(0, len(y_pred)):
        f.write("{0}   {1}   {2}   {3}\n".format(X_test[i,0],X_test[i,1],round(y_pred[i],4),round(np.sqrt(rf_err[i]),4)))

In [7]:
test_data = pd.read_csv("test.txt", delim_whitespace=True)
test_data = test_data[['Z','A']]
test_data = test_data.values.reshape(-1,2)

y_pred_test = rf.predict(test_data)
rf_err_test = abs(fci.random_forest_error(rf,X_train,test_data,calibrate=False))

In [9]:
with open("data_gamma_test.txt","w") as f:
    # f.write(' Z=  22 A=  46\n')
    # f.write('  U[MeV]  fE1[mb/MeV]\n')
    for i in range(0, len(y_pred_test)):
        f.write("{0}   {1}   {2}   {3}\n".format(test_data[i,0],test_data[i,1],round(y_pred_test[i],4),round(np.sqrt(rf_err_test[i]),4)))